# On Local endpolint, Deploy BERT Model
- Tensorflow Serving Model:
    - https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model

In [3]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0

ERROR: tensorflow-text 2.2.1 has requirement tensorflow<2.3,>=2.2.0, but you'll have tensorflow 2.1.0 which is incompatible.
ERROR: tensorflow-serving-api 2.2.0 has requirement tensorflow~=2.2.0, but you'll have tensorflow 2.1.0 which is incompatible.
ERROR: astroid 2.3.3 has requirement wrapt==1.11.*, but you'll have wrapt 1.12.1 which is incompatible.


In [4]:
import tensorflow as tf
tf.version

<module 'tensorflow_core._api.v2.version' from '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow_core/_api/v2/version/__init__.py'>

In [5]:
!pip install -q transformers==2.8.0

In [6]:
!pip install -q sagemaker==1.56.1

## Restart the Kernel to Pick Up New SageMaker Python SDK Library

In [7]:
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [9]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Download the Model to the Notebook

In [10]:
import os

In [11]:
%store -r training_job_name

In [12]:
print(training_job_name)

bert2tweet-2020-07-08-07-58-27-895


In [13]:
model_download = 'model'
os.makedirs(model_download, exist_ok=True)

In [14]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz {model_download}/model.tar.gz

download: s3://sagemaker-us-west-2-057716757052/bert2tweet-2020-07-08-07-58-27-895/output/model.tar.gz to model/model.tar.gz


In [15]:
!tar -xvzf   {model_download}/model.tar.gz

tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/saved_model.pb
transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/assets/


In [16]:
! ls -al {model_download}/

total 1605616
drwxrwxr-x  2 ec2-user ec2-user       4096 Jul  8 08:19 .
drwxrwxr-x 10 ec2-user ec2-user       4096 Jul  8 08:19 ..
-rw-rw-r--  1 ec2-user ec2-user 1644135654 Jul  8 08:11 model.tar.gz


In [18]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

2020-07-08 08:20:39.409854: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:
2020-07-08 08:20:39.409937: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/ef

## Deploy the model

In [24]:
import os
import subprocess

instance_type='ml.m4.xlarge'

# instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = ml.m4.xlarge


In [25]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

In [26]:
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)

endpoint_name = deployed_model.endpoint

print('Endpoint name:  {}'.format(endpoint_name))

Endpoint name:  tensorflow-inference-2020-07-08-08-22-28-901


In [21]:
%store endpoint_name

UsageError: Unknown variable 'endpoint_name'


In [27]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)
    
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes    

In [28]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

In [29]:
import tensorflow as tf
import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 4] This is great!
[Predicted Star Rating: 3] This is not good.
